In [ ]:
import torch, pandas as pd, numpy as np, math
from torch import nn
from torch.utils.data import DataLoader
from Dataset import MyDataset
import warnings
warnings.filterwarnings("ignore")
from train import train
from PeerHTC import PeerHTC
from tqdm import tqdm

In [ ]:
mask = torch.load("BGC/bgc_mask.pt")
fre12 = torch.load("BGC/bgc_fre12.pt")
fre23 = torch.load("BGC/bgc_fre23.pt")

In [ ]:
# for second round of training
# Ad = np.load("BGC/adjacent_matrix.npy")
# weight = np.load("BGC/sample_weight.npy")

In [ ]:
device = "cuda:0"
torch.cuda.manual_seed(1234)
torch.manual_seed(1234)

model = PeerHTC(hidden_dim=128, num_labels=130, num_1=7, num_2=46, num_3=77,
              mask=mask, fre12=fre12, fre23=fre23)

# model.A = nn.Parameter(torch.tensor(Ad, requires_grad=True).float()) # second round

model = model.to(device)

In [ ]:
train_data = MyDataset("BGC/bgc_train_x_10016.pt", "BGC/bgc_train_y_10016.pt", device=device)
# train_data = MyDataset("BGC/bgc_train_x_10016.pt", "BGC/bgc_train_labels&weights_10016.pt", device=device) # second round
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)

val_data = MyDataset("BGC/bgc_val_x.pt", "BGC/bgc_val_y_3.pt", device=device)
val_dataloader = DataLoader(val_data, batch_size=32, shuffle=False)

In [ ]:
train(model, train_dataloader, val_dataloader, 50, 0.0001, 10, "params", 0.5)

In [ ]:
torch.save(model, "modelparams.pt")

In [ ]:
# save probabilities
train_dataloader_2 = DataLoader(train_data, batch_size=32, shuffle=False)
flag = True
with torch.no_grad(): 
    for inputs, labels in tqdm(train_dataloader_2):
        prob = model(inputs)
        if flag:
            probs = prob
            flag = False
        else:
            probs = torch.cat((probs, prob), dim=0)
            
np.save("BGC/train_prob.npy", np.array(probs.cpu()))